In [2]:
import pathlib
import sys
import numpy as np
import pandas as pd
import rasterio
import refet
import pyproj
import xarray
import requests
import multiprocessing as mp
import configparser




In [3]:
#sys.path.append("//")
sys.path.append("../../../Micromet")
import micromet
#from micromet.volk import ffp_climatology as ffp
import micromet.volk as ffp
from micromet import AmerifluxDataProcessor

In [4]:

# load initial flux data
station = 'US-UTW'


In [ ]:


def calc_ffp(station):

    config_path = f'../../station_config/{station}.ini'
    config = configparser.ConfigParser()
    config.read(config_path)

    secrets_path = f'../../secrets/config.ini'
    secrets_config = configparser.ConfigParser()
    secrets_config.read(secrets_path)
    url = secrets_config['DEFAULT']['url']

    startdate = '2022-01-01'

    headdict = {'Accept-Profile': 'groundwater','Content-Type': 'application/json'}

    params = {'stationid':f'eq.{station}',
             'datetime_start':f'gte.{startdate}'
             }
    resp_ed = requests.get(f"{url}/amfluxeddy",headers=headdict,params=params)
    resp_met = requests.get(f"{url}/amfluxmet",headers=headdict,params=params)
    out_dir = pathlib.Path('.')
    df = pd.DataFrame(resp_ed.json())
    df['datetime_start'] = pd.to_datetime(df['datetime_start'])
    df = df.set_index('datetime_start')
    df = df.replace(-9999,np.nan,)
    df = df.resample('1h').mean(numeric_only=True)
    df = df.dropna(subset=['h2o','wd','ustar','v_sigma'])

    latitude = config['METADATA']['station_latitude']
    longitude = config['METADATA']['station_longitude']
    elevation = config['METADATA']['station_elevation']

    station_coord = (latitude, longitude)
    # get EPSG code from lat,long, convert to UTM
    EPSG = 5070
    # move coord to snap centroid to 30m grid, minimal distortion
    transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG:.0f}")
    (station_y, station_x) = transformer.transform(*station_coord)
    print('coordinates:',station_x,station_y)

    #Other model parameters
    h_c = 0.2 #Height of canopy [m]
    d = 10**(0.979* np.log10(h_c) - 0.154) # Estimated displacement height [m]
    zm_s = 2. #Measurement height [m] from AMF metadata
    zm = zm_s - d
    h_s = 2000. #Height of atmos. boundary layer [m] - assumed
    dx = 3. #Model resolution [m]
    origin_d = 200. #Model bounds distance from origin [m]
    # from 7 AM to 8 PM only, modify if needed
    start_hr = 6
    end_hr = 18
    hours = np.arange(start_hr,end_hr+1)
    hours_zero_indexed = np.arange(start_hr-1,end_hr)
    hours_one_indexed = np.arange(start_hr,end_hr+1)
    n_hrs = len(hours_zero_indexed)

    z0 = h_c*.123
    #Loop through each day in the dataframe
    for date in df.index.date:
        #Subset dataframe to only values in day of year
        print(f'Date: {date}')
        temp_df = df[df.index.date == date]
        temp_df = temp_df.between_time(f'{start_hr:02}:00', f'{end_hr:02}:00')
        # check on n hours per day
        if len(temp_df) < 5:
            print(f'Less than 5 hours of data on {date}, skipping.')
            continue

        new_dat = None

        out_f = out_dir/ f'{date}.tif'

        final_outf = out_dir/f'{date.year}-{date.month:02}-{date.day:02}_weighted.tif'

        if final_outf.is_file():
            print(f'final daily weighted footprint already wrote to: {final_outf}\nskipping.')
            continue # do not overwrite date/site raster

        # make hourly band raster for the day
        for indx, hour in enumerate(hours_zero_indexed):

            band = indx + 1
            print(f'Hour: {hour}')

            try:
                temp_line = temp_df.loc[temp_df.index.hour == hour,:]
                if temp_line.empty:
                    print(f'Missing all data for {date,hour} skipping')
                    continue

                #Calculate footprint
                temp_ffp = ffp.ffp_climatology(domain=[-origin_d,origin_d,-origin_d,origin_d],
                               dx=dx,dy=dx,
                               zm=zm, h=h_s, rs=None, z0=z0,
                               ol=temp_line['mo_length'].values,
                               sigmav=temp_line['v_sigma'].values,
                               ustar=temp_line['ustar'].values,
                               umean=temp_line['ws'].values,
                               wind_dir=temp_line['wd'].values,
                               crop=0,fig=0,
                                                   verbosity=4)

                f_2d = np.array(temp_ffp['fclim_2d'])
                x_2d = np.array(temp_ffp['x_2d']) + station_x
                y_2d = np.array(temp_ffp['y_2d']) + station_y
                f_2d = f_2d*dx**2

                #Calculate affine transform for given x_2d and y_2d
                affine_transform = micromet.find_transform(x_2d,y_2d)

                #Create data file if not already created
                if new_dat is None:
                    #print(f_2d.shape)
                    new_dat = rasterio.open(
                        out_f,'w',
                        driver='GTiff',
                        dtype=rasterio.float64,
                        count=n_hrs,
                        height=f_2d.shape[0],
                        width=f_2d.shape[1],
                        transform=affine_transform,
                        crs=transformer.target_crs,
                        nodata=0.00000000e+000
                    )

            except Exception as e:
                print(e)
                print(f'Hour {hour} footprint failed, band {band} not written.')

                temp_ffp = None

                continue

            #Mask out points that are below a % threshold (defaults to 90%)
            f_2d = ffp.mask_fp_cutoff(f_2d)

            #Write the new band
            new_dat.write(f_2d, indx+1)

            #Update tags with metadata
            tag_dict = {'hour':f'{hour*100:04}',
                        'wind_dir':temp_line['wd'].values,
                        'total_footprint':np.nansum(f_2d)}

            new_dat.update_tags(indx+1,**tag_dict)

        #Close dataset if it exists
        try:
            new_dat.close()
        except:
            print(f'ERROR: could not write footprint for site: {station}:\nto: {out_f}')
            continue # skip to next day...

pool = mp.Pool(processes=8)
pool.map(calc_ffp,'US-UTW')

In [32]:
eto_df = pd.read_parquet('nldas_all_normed.parquet')
eto_df['daily_ETo_normed'] = eto_df['daily_ETo_normed'].fillna(0)
stationid = 'US-UTW'
nldas_df = eto_df.loc[stationid]
start_hr = 6
end_hr = 18

out_dir = pathlib.Path('.')
for out_f in out_dir.glob('20*.tif'):
    if 'weighted' in str(out_f.stem):
        pass
    else:
        print(out_f)

        date = pd.to_datetime(out_f.stem, format='%Y-%m-%d')
        src = rasterio.open(out_f)
        hours = np.array(src.indexes)+start_hr-1
        final_outf = out_dir/f'{date.year}-{date.month:02}-{date.day:02}_weighted.tif'

        if final_outf.is_file():
            print(f'final daily weighted footprint already wrote to: {final_outf}\nskipping.')
            continue # do not overwrite date/site raster

        # do hourly weighting - do not necessarily need to do this all in the same loop
        src = rasterio.open(out_f)
        n_hrs = len(src.indexes)
        # hourly fetch scalar sums
        global_sum = np.zeros(shape=len(src.indexes))
        # normalized fetch rasters
        normed_fetch_rasters = []
        for ind in src.indexes:
            hour = ind+start_hr-1
            dtindex = pd.to_datetime(f"{date:%Y-%m-%d} {hour:02d}:00:00",
                                     format='%Y-%m-%d %H:%M:%S')
            print(dtindex, ind, hour)
            norm_eto = nldas_df.loc[dtindex,'daily_ETo_normed']
            arr = src.read(ind)
            global_sum = arr.sum()
            tmp = arr / global_sum
            if np.isnan(tmp).all():
                tmp = np.zeros_like(tmp)
            normed_fetch_rasters.append(tmp*norm_eto)
            # Last calculation, sum the weighted hourly rasters to a single daily fetch raster
        final_footprint = sum(normed_fetch_rasters)

        if np.isclose(final_footprint.sum(), 1, atol=0.15):

            # finally, write daily corrected raster with UTM zone reference
            corr_raster_path = final_outf
            out_raster = rasterio.open(
                corr_raster_path,'w',driver='GTiff',dtype=rasterio.float64,
                count=1,height=final_footprint.shape[0],width=final_footprint.shape[1],
                transform=src.transform, crs=src.crs, nodata=0.00000000e+000
            )
            out_raster.write(final_footprint,1)
            out_raster.close()
        else:
            print(f'check 1 failed! {final_footprint.sum()}\n{dtindex}')

2022-01-13.tif
final daily weighted footprint already wrote to: 2022-01-13_weighted.tif
skipping.
2022-01-14.tif
final daily weighted footprint already wrote to: 2022-01-14_weighted.tif
skipping.
2022-01-15.tif
final daily weighted footprint already wrote to: 2022-01-15_weighted.tif
skipping.
2022-01-16.tif
final daily weighted footprint already wrote to: 2022-01-16_weighted.tif
skipping.
2022-01-17.tif
final daily weighted footprint already wrote to: 2022-01-17_weighted.tif
skipping.
2022-01-18.tif
final daily weighted footprint already wrote to: 2022-01-18_weighted.tif
skipping.
2022-01-19.tif
final daily weighted footprint already wrote to: 2022-01-19_weighted.tif
skipping.
2022-01-20.tif
final daily weighted footprint already wrote to: 2022-01-20_weighted.tif
skipping.
2022-01-21.tif
final daily weighted footprint already wrote to: 2022-01-21_weighted.tif
skipping.
2022-01-22.tif
final daily weighted footprint already wrote to: 2022-01-22_weighted.tif
skipping.
2022-01-23.tif
final

C:\Users\paulinkenbrandt\AppData\Local\Temp\1\ipykernel_28476\3628216221.py:39: RuntimeWarning: invalid value encountered in divide
  tmp = arr / global_sum


2022-04-08 07:00:00 2 7
2022-04-08 08:00:00 3 8
2022-04-08 09:00:00 4 9
2022-04-08 10:00:00 5 10
2022-04-08 11:00:00 6 11
2022-04-08 12:00:00 7 12
2022-04-08 13:00:00 8 13
2022-04-08 14:00:00 9 14
2022-04-08 15:00:00 10 15
2022-04-08 16:00:00 11 16
2022-04-08 17:00:00 12 17
2022-04-08 18:00:00 13 18
2022-04-09.tif
2022-04-09 06:00:00 1 6
2022-04-09 07:00:00 2 7
2022-04-09 08:00:00 3 8
2022-04-09 09:00:00 4 9
2022-04-09 10:00:00 5 10
2022-04-09 11:00:00 6 11
2022-04-09 12:00:00 7 12
2022-04-09 13:00:00 8 13
2022-04-09 14:00:00 9 14
2022-04-09 15:00:00 10 15
2022-04-09 16:00:00 11 16
2022-04-09 17:00:00 12 17
2022-04-09 18:00:00 13 18
2022-04-10.tif
2022-04-10 06:00:00 1 6
2022-04-10 07:00:00 2 7
2022-04-10 08:00:00 3 8
2022-04-10 09:00:00 4 9
2022-04-10 10:00:00 5 10
2022-04-10 11:00:00 6 11
2022-04-10 12:00:00 7 12
2022-04-10 13:00:00 8 13
2022-04-10 14:00:00 9 14
2022-04-10 15:00:00 10 15
2022-04-10 16:00:00 11 16
2022-04-10 17:00:00 12 17
2022-04-10 18:00:00 13 18
2022-04-11.tif
2022